<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/Removed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Removed

Rimpiazzo i NaN con il valore 0 (da rivedere con le tecniche per rimpiazzare i missing values. 

Faccio il one hot encoding per avere solamente feature numeriche. (anche questo può essere riconsiderato)

In [ ]:
# elimino i nan andandoli a rimpiazzare con zero 
data = data.replace(to_replace = np.nan, value = 0)

# faccio il one hot encoding delle feature categoriche così da avere solo valori numerici
data = pd.get_dummies(data)

In [ ]:
sec_in_day = 86400
# tempo da cui inizio a selezionare
start = sec_in_day - 1
# tempo in cui si conclude lo split 
end = sec_in_day * 2 - 1
# creo un array che contiene i dati suddivisi per giorno
sub_data = []
# for i in range(int(total_days)):
for i in range(10):
#   per concatenare due condizioni devo delimitarle con le parentesi 
#   https://stackoverflow.com/questions/31617845/how-to-select-rows-in-a-dataframe-between-two-values-in-python-pandas
    sub_data.append(data[(data['TransactionDT'] < end) & (data['TransactionDT'] > start)]);
    start += sec_in_day
    end += sec_in_day
# sub_data

fraud = []
safe = []
fraud_distr = []
for i in range(9):
    fraud_distr.append(sub_data[i][['TransactionDT','TransactionAmt','isFraud']]) 
    fraud.append( fraud_distr[i][fraud_distr[i]['isFraud'] == 1])
    safe.append(fraud_distr[i][fraud_distr[i]['isFraud'] == 0])

x = []
y = []
# https://matplotlib.org/3.3.2/gallery/subplots_axes_and_figures/subplot.html#sphx-glr-gallery-subplots-axes-and-figures-subplot-py
# https://stackoverflow.com/questions/17210646/python-subplot-within-a-loop-first-panel-appears-in-wrong-position
fig, ax = plt.subplots(3, 3, figsize = (30,10))
fig.subplots_adjust(wspace = 1)
ax = ax.ravel()
# for loop per plottare i vari subplots
for i in range(9):
    x.append([fraud[i]['TransactionDT'], safe[i]['TransactionDT']])
    y.append([fraud[i]['TransactionAmt'], safe[i]['TransactionAmt']])

    fig.suptitle('Amount nel tempo')

    ax[i].plot(x[i][0], y[i][0], label = 'fraud')
    ax[i].plot( x[i][1],  y[i][1], alpha = 0.5, label = 'safe')
#     ax[i].set_title('Day')
#     ax[i].set_ylabel('amount')
    ax[i].set_xlabel('time (s)')

plt.show()

dev_info = original_data[['DeviceInfo', 'isFraud']]
dev_info = dev_info[dev_info['DeviceInfo'] != 0]
dev_info_fraud = dev_info[dev_info['isFraud'] == 1].value_counts(normalize = True)
dev_info_safe = dev_info[dev_info['isFraud'] == 0].value_counts(normalize = True)

dev_info_fraud[:10].plot(kind = 'bar', label = 'fraud', color = 'red');
dev_info_safe[:10].plot(kind = 'bar',  label='safe', color = 'green');


#### Altro modo (peggiore) per fare l'iterative imputer su tutto il dataset
Faccio l'iterative imputer su tutto il dataset però tenendo solo la colonna transactionAmt -> più veloce ma non è comunque corretto dal punto di vista concettuale. L'algoritmo stima i valori mancanti attraverso tutti gli altri valori contenuti nel dataset. Se gli dessi solo una colonna, potrebbe non essere sufficiente per valutare bene i missing values. 

Creo un dizionario (fitted_set) che conterrà le colonne con i valori stimati. 

Faccio un loop su tutte le colonne per stimare i valori. 

Con round setto il numero di decimali. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.round.html

fitted è il dataframe che contiene i valori rimpiazzati, quindi senza NaN.

In [ ]:
do = False

if do:
    # dizionario che contiene le colonne con i valori rimpiazzati
    fitted_set = {}

    for col in num_data.columns:
        if col != 'TransactionAmt':
            subset = num_data[['TransactionAmt',col]]
            imp = IterativeImputer(missing_values=np.nan, random_state=0, n_nearest_features=5)                          
            imp.fit(subset)
            subset = imp.transform(subset)
            fitted_set[col] = subset[:,1]

    fitted = pd.DataFrame(fitted_set).round(2)
    del fitted_set
    # fitted

## Correlazione senza rimpiazzo dei missing values 
Cerco correlazioni tra le features. 
Vado ad eliminare le colonne con troppi zeri che sono meno informative. 

In [1]:
do = False 
if do: 
    data_splitted = split_by_day(original_data, 1)

    # dizionario che contengono la matrice con i feature vector
    X = {}
    # dizionario che contiene le label
    y = {}

    # popolo X e y
    for i in range(n_set):
        X[i] = data_splitted[i].drop(['isFraud'], axis = 1)
        y[i] = data_splitted[i]['isFraud']

    # dizionario con i vari dataset per la correlazione
    X_correlation = {}
    thresh = 2000
    sub_dataset = 0
    # https://stackoverflow.com/questions/46628253/deleting-dataframe-column-in-pandas-based-on-value
    # https://stackoverflow.com/questions/53550988/count-occurrences-of-false-or-true-in-a-column-in-pandas
    X_correlation[sub_dataset] = X[sub_dataset].loc[:, ((X[sub_dataset] != 0).sum() > thresh)]
    # X_correlation[sub_dataset]
    
    # https://stackoverflow.com/questions/29432629/plot-correlation-matrix-using-pandas
    f = plt.figure(figsize=(40, 40))
    plt.matshow(X_correlation[sub_dataset].corr(), fignum=f.number)
    plt.xticks(range(X_correlation[sub_dataset].shape[1]), X_correlation[sub_dataset].columns, fontsize=10, rotation=90)
    plt.yticks(range(X_correlation[sub_dataset].shape[1]), X_correlation[sub_dataset].columns, fontsize=10)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title('Correlation Matrix', fontsize=16);